In [411]:
import pandas as pd
import numpy as np
import datetime
import math
import sklearn.cross_validation
from scipy import stats

In [412]:
train = pd.read_csv('data_original/train.csv')
test = pd.read_csv('data_original/test.csv')

print(len(train.Point.unique()))
print(len(test.Point.unique()))
a = set(train.Point.unique()) - (set(train.Point.unique()) & set(test.Point.unique()))
for i in a:
    print(i)
    train = train[train.Point != i]

    
test['Count'] = None
frame = [train, test]
common = pd.concat(frame)
common.Date = common.Date.apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y'))
common = common.sort_values(by=['Date'])

114
114


In [413]:
common

,Date,Point,Count
6712,2011-10-06,Сыктывкар-1,8.0
39414,2011-10-07,Сыктывкар-1,88.0
4141,2011-10-09,Сыктывкар-1,1.0
832,2011-11-15,Сыктывкар-1,1.0
23476,2011-12-08,Сыктывкар-1,2.0
11001,2011-12-09,Сыктывкар-1,43.0
40810,2011-12-10,Сыктывкар-1,10.0
44315,2011-12-11,Сыктывкар-1,1.0
22002,2011-12-13,Сыктывкар-1,60.0
6805,2011-12-14,Сыктывкар-1,NaN


In [414]:
common.to_csv('data_original/common.csv', index=None)
train.to_csv('data_original/train.csv', index=None)
test.to_csv('data_original/test.csv', index=None)

# date.csv

In [415]:
date = pd.read_csv('data_original/common.csv')[['Date']]
date.Date = date.Date.apply(lambda x: pd.to_datetime(x))

In [416]:
start_date = date.iloc[0]['Date']
date['Year'] = date.Date.apply(lambda x: x.year)
date['Month'] = date.Date.apply(lambda x: x.month)
date['MonthDay'] = date.Date.apply(lambda x: x.day)
date['WeekDay'] = date.Date.apply(lambda x: x.weekday())
date['Point'] = pd.Series(common.Point.as_matrix())

In [417]:
date['YearDay'] = date.Date.apply(lambda x: (x-pd.to_datetime(str(x.year)+'-01-01')).days)

In [418]:
date['ThroughMonth'] = date.Date.apply(lambda x: (x.year - start_date.year) * 12 + x.month - start_date.month)

In [419]:
date['ThroughWeek'] = date.Date.apply(lambda x: int( ( (x - start_date).days + start_date.weekday()) / 7))

In [420]:
date['ThroughDay'] = date.Date.apply(lambda x: (x - start_date).days)

In [421]:
date.to_csv('data_transform/date.csv', index=None)

In [422]:
date.head(10)

,Date,Year,Month,MonthDay,WeekDay,Point,YearDay,ThroughMonth,ThroughWeek,ThroughDay
0,2011-10-06,2011,10,6,3,Сыктывкар-1,278,0,0,0
1,2011-10-07,2011,10,7,4,Сыктывкар-1,279,0,0,1
2,2011-10-09,2011,10,9,6,Сыктывкар-1,281,0,0,3
3,2011-11-15,2011,11,15,1,Сыктывкар-1,318,1,6,40
4,2011-12-08,2011,12,8,3,Сыктывкар-1,341,2,9,63
5,2011-12-09,2011,12,9,4,Сыктывкар-1,342,2,9,64
6,2011-12-10,2011,12,10,5,Сыктывкар-1,343,2,9,65
7,2011-12-11,2011,12,11,6,Сыктывкар-1,344,2,9,66
8,2011-12-13,2011,12,13,1,Сыктывкар-1,346,2,10,68
9,2011-12-14,2011,12,14,2,Сыктывкар-1,347,2,10,69


# target.csv

In [423]:
sizeValidationSample = 0.16

In [424]:
target = pd.read_csv('data_original/common.csv')[['Count', 'Date']]
#target['Point'] = pd.Series(common.Point.as_matrix())
target['IsTest'] = target.Count
target['IsTrain'] = pd.Series(range(target.shape[0]))
target['IsValidation'] = pd.Series(range(target.shape[0]))

In [425]:
target['IsTest'] = target.IsTest.apply(lambda x: 1 if math.isnan(x) else 0)

In [426]:
withCountValue = target[target.IsTest == 0].index
indexTrain, indexValidation = sklearn.cross_validation.train_test_split(withCountValue, test_size=sizeValidationSample)

In [427]:
dates = target.Date[target.IsTest == 1]
test_date_count = dates.unique().shape[0]
print(test_date_count)
dates = target.Date[target.IsTest == 0].unique()
np.random.shuffle(dates)
validation_dates = dates[:test_date_count]
validation_dates
target['IsValidation'] = target.Date.apply(lambda x: 1 if x in validation_dates else 0)
target

268


,Count,Date,IsTest,IsTrain,IsValidation
0,8.0,2011-10-06,0,0,1
1,88.0,2011-10-07,0,1,0
2,1.0,2011-10-09,0,2,0
3,1.0,2011-11-15,0,3,0
4,2.0,2011-12-08,0,4,0
5,43.0,2011-12-09,0,5,0
6,10.0,2011-12-10,0,6,0
7,1.0,2011-12-11,0,7,1
8,60.0,2011-12-13,0,8,0
9,NaN,2011-12-14,1,9,0


In [428]:
target['IsTrain'] = 1
indexes = target[np.logical_or(target.IsValidation == 1, target.IsTest == 1)].index
target.IsTrain.loc[indexes] = 0
#target['IsValidation'] = target.IsValidation.apply(lambda x: 1 if ((x in indexValidation) & (target.IsValidation[x] == 0)) else 0)

/opt/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [429]:
target.to_csv('data_transform/target.csv', index = None)

In [430]:
CountWithouthNaNValueValidation = target.Count.copy()

In [431]:
target.Count.loc[target.Count[target.IsValidation == 1].index] = np.nan


/opt/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [432]:
target.head(20)

,Count,Date,IsTest,IsTrain,IsValidation
0,NaN,2011-10-06,0,0,1
1,88.0,2011-10-07,0,1,0
2,1.0,2011-10-09,0,1,0
3,1.0,2011-11-15,0,1,0
4,2.0,2011-12-08,0,1,0
5,43.0,2011-12-09,0,1,0
6,10.0,2011-12-10,0,1,0
7,NaN,2011-12-11,0,0,1
8,60.0,2011-12-13,0,1,0
9,NaN,2011-12-14,1,0,0


In [433]:
target

,Count,Date,IsTest,IsTrain,IsValidation
0,NaN,2011-10-06,0,0,1
1,88.0,2011-10-07,0,1,0
2,1.0,2011-10-09,0,1,0
3,1.0,2011-11-15,0,1,0
4,2.0,2011-12-08,0,1,0
5,43.0,2011-12-09,0,1,0
6,10.0,2011-12-10,0,1,0
7,NaN,2011-12-11,0,0,1
8,60.0,2011-12-13,0,1,0
9,NaN,2011-12-14,1,0,0


# city.csv


In [434]:
cities = pd.read_csv('data_original/common.csv')[['Point']].Point.unique()
city = pd.DataFrame()
city['Point'] = pd.Series(cities)
city['CityName'] = city.Point.apply(lambda x: '-'.join(x.split('-')[0:-1]))
city_names = city.CityName.unique()
city['CityID'] = city.CityName.apply(lambda x: np.where(city_names == x)[0][0]) 
city['BranchNumber'] = city.Point.apply(lambda x: x.split('-')[-1:][0])
# Москва 1 -> Москва
city['CityName'] = city.CityName.apply(lambda x: ('Москва' if x == 'Москва 1' else x))
city.to_csv('data_transform/city.csv', index = None)

In [435]:
city.head(10)

,Point,CityName,CityID,BranchNumber
0,Сыктывкар-1,Сыктывкар,0,1
1,Сыктывкар-2,Сыктывкар,0,2
2,Ухта-1,Ухта,1,1
3,Смоленск-1,Смоленск,2,1
4,Сарапул-1,Сарапул,3,1
5,Самара-1,Самара,4,1
6,Челябинск-1,Челябинск,5,1
7,Белгород-1,Белгород,6,1
8,Смоленск-2,Смоленск,2,2
9,Самара-2,Самара,4,2


# internal_features.csv

In [436]:
src = pd.read_csv('data_transform/date.csv')
src['Point'] = pd.read_csv('data_original/common.csv')['Point']
src['Count'] = target.Count
dst = pd.DataFrame() 

In [437]:
src.head(15)

,Date,Year,Month,MonthDay,WeekDay,Point,YearDay,ThroughMonth,ThroughWeek,ThroughDay,Count
0,2011-10-06,2011,10,6,3,Сыктывкар-1,278,0,0,0,NaN
1,2011-10-07,2011,10,7,4,Сыктывкар-1,279,0,0,1,88.0
2,2011-10-09,2011,10,9,6,Сыктывкар-1,281,0,0,3,1.0
3,2011-11-15,2011,11,15,1,Сыктывкар-1,318,1,6,40,1.0
4,2011-12-08,2011,12,8,3,Сыктывкар-1,341,2,9,63,2.0
5,2011-12-09,2011,12,9,4,Сыктывкар-1,342,2,9,64,43.0
6,2011-12-10,2011,12,10,5,Сыктывкар-1,343,2,9,65,10.0
7,2011-12-11,2011,12,11,6,Сыктывкар-1,344,2,9,66,NaN
8,2011-12-13,2011,12,13,1,Сыктывкар-1,346,2,10,68,60.0
9,2011-12-14,2011,12,14,2,Сыктывкар-1,347,2,10,69,NaN


## Next-Prev N Value

In [438]:
def GetValueForNPrevNextDay(n, target, src):
    targetCopy = target.copy()
    targetCopy.Count.loc[targetCopy.IsValidation == 1] = np.nan

    targetCopy['Point'] = src.Point

    city = src.Point.unique()

    result = np.array([np.nan] * targetCopy.shape[0])
    for i in city:
        value = targetCopy.Count[targetCopy.Point == i].shift(n)
        for j in value.index:
            result[j] = value[j]


    prefics = "Prev" if n > 0 else "Next"
    targetCopy[prefics + str(abs(n)) + "Day"] = pd.Series(result)
    del targetCopy["Point"]
    return targetCopy[prefics + str(abs(n)) + "Day"]

In [439]:
#target["Prev1Day"] = GetValueForNPrevNextDay(1, target, src)
#target["Prev2Day"] = GetValueForNPrevNextDay(2, target, src)
#target["Prev3Day"] = GetValueForNPrevNextDay(3, target, src)
#target["Prev7Day"] = GetValueForNPrevNextDay(7, target, src)

In [440]:
#target["Next1Day"] = GetValueForNPrevNextDay(-1, target, src)
#target["Next2Day"] = GetValueForNPrevNextDay(-2, target, src)
#target["Next3Day"] = GetValueForNPrevNextDay(-3, target, src)
#target["Next7Day"] = GetValueForNPrevNextDay(-7, target, src)

In [441]:
#target.to_csv('data_transform/target.csv', index = None)

In [442]:
target.head(20)

,Count,Date,IsTest,IsTrain,IsValidation
0,NaN,2011-10-06,0,0,1
1,88.0,2011-10-07,0,1,0
2,1.0,2011-10-09,0,1,0
3,1.0,2011-11-15,0,1,0
4,2.0,2011-12-08,0,1,0
5,43.0,2011-12-09,0,1,0
6,10.0,2011-12-10,0,1,0
7,NaN,2011-12-11,0,0,1
8,60.0,2011-12-13,0,1,0
9,NaN,2011-12-14,1,0,0


## Mean

### by date independed from points

In [443]:
def calc_mean(src, group_column, target_column):
    mean = src.groupby([group_column])[target_column].mean()
    return src[group_column].apply(lambda x: mean[x])

In [444]:
need_mean_list = src.columns.difference(['Date', 'Count', 'Point'])
for column in need_mean_list:
    dst['MeanFor' + column] = calc_mean(src, column, 'Count')

In [445]:
dst.head(15)

,MeanForMonth,MeanForMonthDay,MeanForThroughDay,MeanForThroughMonth,MeanForThroughWeek,MeanForWeekDay,MeanForYear,MeanForYearDay
0,112.169891,104.965901,NaN,44.500000,44.500000,91.609235,90.222222,95.668966
1,112.169891,106.206302,88.0,44.500000,44.500000,122.180506,90.222222,89.567416
2,112.169891,105.964580,1.0,44.500000,44.500000,123.046591,90.222222,106.166667
3,100.172292,112.235336,1.0,1.000000,1.000000,87.417214,90.222222,126.147727
4,104.290971,107.566723,2.0,102.266667,18.333333,91.609235,90.222222,76.451220
5,104.290971,105.964580,43.0,102.266667,18.333333,122.180506,90.222222,76.759615
6,104.290971,105.938329,10.0,102.266667,18.333333,143.187179,90.222222,77.217391
7,104.290971,105.069268,NaN,102.266667,18.333333,123.046591,90.222222,105.086022
8,104.290971,99.680982,60.0,102.266667,100.500000,87.417214,90.222222,124.409639
9,104.290971,103.970904,NaN,102.266667,100.500000,87.628759,90.222222,160.363636


### by date depended from points

In [446]:
def map_group_two_key(group, x, y):
    return group[x][y]

def calc_mean_by_columns(src, group_columns, target_column):
    mean = src.groupby(group_columns)[target_column].mean()
    return src[group_columns].apply(lambda x: map_group_two_key(mean, *x), axis=1)



In [447]:
need_list = src.columns.difference(['Date', 'Count', 'Point'])

In [448]:
for column in need_list:
    dst['MeanFor' + column + 'ByPoints'] = calc_mean_by_columns(src, [column, 'Point'], 'Count')

In [449]:
dst['Point'] = pd.read_csv('data_original/common.csv')['Point']
dst.to_csv('data_transform/internal_features.csv', index=None)

In [450]:
dst

,MeanForMonth,MeanForMonthDay,MeanForThroughDay,MeanForThroughMonth,MeanForThroughWeek,MeanForWeekDay,MeanForYear,MeanForYearDay,MeanForMonthByPoints,MeanForMonthDayByPoints,MeanForThroughDayByPoints,MeanForThroughMonthByPoints,MeanForThroughWeekByPoints,MeanForWeekDayByPoints,MeanForYearByPoints,MeanForYearDayByPoints,Point
0,112.169891,104.965901,NaN,44.500000,44.500000,91.609235,90.222222,95.668966,267.826923,231.285714,NaN,44.500000,44.500000,213.406593,90.222222,272.500000,Сыктывкар-1
1,112.169891,106.206302,88.0,44.500000,44.500000,122.180506,90.222222,89.567416,267.826923,254.454545,88.0,44.500000,44.500000,287.532967,90.222222,185.250000,Сыктывкар-1
2,112.169891,105.964580,1.0,44.500000,44.500000,123.046591,90.222222,106.166667,267.826923,232.634146,1.0,44.500000,44.500000,262.294118,90.222222,204.250000,Сыктывкар-1
3,100.172292,112.235336,1.0,1.000000,1.000000,87.417214,90.222222,126.147727,250.382716,235.822222,1.0,1.000000,1.000000,198.060773,90.222222,190.250000,Сыктывкар-1
4,104.290971,107.566723,2.0,102.266667,18.333333,91.609235,90.222222,76.451220,235.745098,249.121951,2.0,102.266667,18.333333,213.406593,90.222222,190.750000,Сыктывкар-1
5,104.290971,105.964580,43.0,102.266667,18.333333,122.180506,90.222222,76.759615,235.745098,232.634146,43.0,102.266667,18.333333,287.532967,90.222222,183.800000,Сыктывкар-1
6,104.290971,105.938329,10.0,102.266667,18.333333,143.187179,90.222222,77.217391,235.745098,221.853659,10.0,102.266667,18.333333,317.926966,90.222222,73.500000,Сыктывкар-1
7,104.290971,105.069268,NaN,102.266667,18.333333,123.046591,90.222222,105.086022,235.745098,234.216216,NaN,102.266667,18.333333,262.294118,90.222222,217.666667,Сыктывкар-1
8,104.290971,99.680982,60.0,102.266667,100.500000,87.417214,90.222222,124.409639,235.745098,209.485714,60.0,102.266667,100.500000,198.060773,90.222222,197.750000,Сыктывкар-1
9,104.290971,103.970904,NaN,102.266667,100.500000,87.628759,90.222222,160.363636,235.745098,243.195122,NaN,102.266667,100.500000,199.602339,90.222222,374.000000,Сыктывкар-1


# All in one

In [451]:
data = pd.read_csv('data_transform/target.csv')

In [452]:
data

,Count,Date,IsTest,IsTrain,IsValidation
0,8.0,2011-10-06,0,0,1
1,88.0,2011-10-07,0,1,0
2,1.0,2011-10-09,0,1,0
3,1.0,2011-11-15,0,1,0
4,2.0,2011-12-08,0,1,0
5,43.0,2011-12-09,0,1,0
6,10.0,2011-12-10,0,1,0
7,1.0,2011-12-11,0,0,1
8,60.0,2011-12-13,0,1,0
9,NaN,2011-12-14,1,0,0


In [453]:
date = pd.read_csv('data_transform/date.csv')
city = pd.read_csv('data_transform/city.csv')
internal_features = pd.read_csv('data_transform/internal_features.csv')

need_columns = date.columns
data[need_columns] = date[need_columns]

data = pd.merge(data, city, on='Point', how='left')

need_columns = internal_features.columns
data[need_columns] = internal_features[need_columns]

#data = pd.merge(data, dollar, on='Date', how='left')

data.to_csv('data_transform/all.csv', index=None)

In [454]:
pd.isnull(data).sum() > 0

Count                           True
Date                           False
IsTest                         False
IsTrain                        False
IsValidation                   False
Year                           False
Month                          False
MonthDay                       False
WeekDay                        False
Point                          False
YearDay                        False
ThroughMonth                   False
ThroughWeek                    False
ThroughDay                     False
CityName                       False
CityID                         False
BranchNumber                   False
MeanForMonth                   False
MeanForMonthDay                False
MeanForThroughDay               True
MeanForThroughMonth            False
MeanForThroughWeek             False
MeanForWeekDay                 False
MeanForYear                    False
MeanForYearDay                  True
MeanForMonthByPoints            True
MeanForMonthDayByPoints         True
M

In [455]:
data

,Count,Date,IsTest,IsTrain,IsValidation,Year,Month,MonthDay,WeekDay,Point,...,MeanForYear,MeanForYearDay,MeanForMonthByPoints,MeanForMonthDayByPoints,MeanForThroughDayByPoints,MeanForThroughMonthByPoints,MeanForThroughWeekByPoints,MeanForWeekDayByPoints,MeanForYearByPoints,MeanForYearDayByPoints
0,8.0,2011-10-06,0,0,1,2011,10,6,3,Сыктывкар-1,...,90.222222,95.668966,267.826923,231.285714,NaN,44.500000,44.500000,213.406593,90.222222,272.500000
1,88.0,2011-10-07,0,1,0,2011,10,7,4,Сыктывкар-1,...,90.222222,89.567416,267.826923,254.454545,88.0,44.500000,44.500000,287.532967,90.222222,185.250000
2,1.0,2011-10-09,0,1,0,2011,10,9,6,Сыктывкар-1,...,90.222222,106.166667,267.826923,232.634146,1.0,44.500000,44.500000,262.294118,90.222222,204.250000
3,1.0,2011-11-15,0,1,0,2011,11,15,1,Сыктывкар-1,...,90.222222,126.147727,250.382716,235.822222,1.0,1.000000,1.000000,198.060773,90.222222,190.250000
4,2.0,2011-12-08,0,1,0,2011,12,8,3,Сыктывкар-1,...,90.222222,76.451220,235.745098,249.121951,2.0,102.266667,18.333333,213.406593,90.222222,190.750000
5,43.0,2011-12-09,0,1,0,2011,12,9,4,Сыктывкар-1,...,90.222222,76.759615,235.745098,232.634146,43.0,102.266667,18.333333,287.532967,90.222222,183.800000
6,10.0,2011-12-10,0,1,0,2011,12,10,5,Сыктывкар-1,...,90.222222,77.217391,235.745098,221.853659,10.0,102.266667,18.333333,317.926966,90.222222,73.500000
7,1.0,2011-12-11,0,0,1,2011,12,11,6,Сыктывкар-1,...,90.222222,105.086022,235.745098,234.216216,NaN,102.266667,18.333333,262.294118,90.222222,217.666667
8,60.0,2011-12-13,0,1,0,2011,12,13,1,Сыктывкар-1,...,90.222222,124.409639,235.745098,209.485714,60.0,102.266667,100.500000,198.060773,90.222222,197.750000
9,NaN,2011-12-14,1,0,0,2011,12,14,2,Сыктывкар-1,...,90.222222,160.363636,235.745098,243.195122,NaN,102.266667,100.500000,199.602339,90.222222,374.000000
